# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,819.486726,-0.431945,1234,819.054780,819.918671,1638.109561,1639.837341
6,325.410466,0.841517,1234,326.251983,324.568949,652.503966,649.137897
7,552.207248,0.384309,1234,552.591557,551.822939,1105.183114,1103.645878
8,300.294549,-0.596069,1234,299.698480,300.890619,599.396960,601.781237
9,999.154308,-0.036273,1234,999.118035,999.190580,1998.236069,1998.381161
10,505.854994,0.390441,1234,506.245435,505.464553,1012.490869,1010.929105
11,344.181221,0.703568,1234,344.884790,343.477653,689.769579,686.955306
12,308.401156,-0.815717,1234,307.585439,309.216873,615.170878,618.433746
13,742.565500,-0.535609,1234,742.029890,743.101109,1484.059781,1486.202218
14,185.339347,-0.564205,1234,184.775141,185.903552,369.550282,371.807104


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-819.054780,819.918671
6,-326.251983,324.568949
7,-552.591557,551.822939
8,-299.698480,300.890619
9,-999.118035,999.190580
10,-506.245435,505.464553
11,-344.884790,343.477653
12,-307.585439,309.216873
13,-742.029890,743.101109
14,-184.775141,185.903552


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-819.054780,819.918671
6,-326.251983,324.568949
7,-552.591557,551.822939
8,-299.698480,300.890619
9,-999.118035,999.190580
10,-506.245435,505.464553
11,-344.884790,343.477653
12,-307.585439,309.216873
13,-742.029890,743.101109
14,-184.775141,185.903552


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-819.05478047,  819.91867072],
       [-326.25198277,  324.56894861],
       [-552.59155707,  551.82293907],
       [-299.69847979,  300.89061864],
       [-999.11803469,  999.1905804 ],
       [-506.24543461,  505.46455269],
       [-344.88478965,  343.47765279],
       [-307.58543899,  309.21687313],
       [-742.02989033,  743.10110902],
       [-184.77514109,  185.90355197]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1636.109561,-1639.837341
6,650.503966,-649.137897
7,1103.183114,-1103.645878
8,597.396960,-601.781237
9,1996.236069,-1998.381161
10,1010.490869,-1010.929105
11,687.769579,-686.955306
12,613.170878,-618.433746
13,1482.059781,-1486.202218
14,367.550282,-371.807104


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.819487,0.284027
6,0.325410,0.920759
7,0.552207,0.692154
8,0.300295,0.201965
9,0.999154,0.481864
10,0.505855,0.695220
11,0.344181,0.851784
12,0.308401,0.092141
13,0.742565,0.232195
14,0.185339,0.217897


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,819.486726,-0.431945
6,325.410466,0.841517
7,552.207248,0.384309
8,300.294549,-0.596069
9,999.154308,-0.036273
10,505.854994,0.390441
11,344.181221,0.703568
12,308.401156,-0.815717
13,742.565500,-0.535609
14,185.339347,-0.564205


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
